# Trabalho Final

**Aluno**: Lucas de Miranda Rodrigues

**Matrícula**: 96143


## 04 - Tratamento da base *habilidades.csv*

A base habilidades descreve as skills de cada perfil. Tem ao todo 100 perfis nessa base, mas 111 é quantidade total de perfis. E o número total de registros na base são de 2594, podendo, então, perfis aparacerem mais deu uma vez na  base.

Essa base tem a coluna das quantidades de endosso (**endorsement_count**) para cada skill, que está na coluna **name**.


In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

habilidades = pd.read_csv("bases/habilidades.csv")

# Análise inicial
print("Nº de registros:", len(habilidades))
print("Nº de valores únicos de perfil_id:", habilidades.perfil_id.nunique())
print("Nº de valores únicos de name:", habilidades.name.nunique())
print("Valores faltantes em name:", habilidades.name.isnull().sum())
print("Valores faltantes em endorsement_count:", habilidades.endorsement_count.isnull().sum())

habilidades.describe(include="all")

Nº de registros: 2594
Nº de valores únicos de perfil_id: 100
Nº de valores únicos de name: 764
Valores faltantes em name: 417
Valores faltantes em endorsement_count: 618


,endorsement_count,name,perfil_id
count,1976,2177,2594.000000
unique,69,764,NaN
top,2,Microsoft Office,NaN
freq,297,35,NaN
mean,NaN,NaN,55.196993
std,NaN,NaN,31.803147
min,NaN,NaN,0.000000
25%,NaN,NaN,31.000000
50%,NaN,NaN,56.000000
75%,NaN,NaN,82.000000


Apesar da grande quantidade de registros, os registros sem **name** podem ser desconsiderados pois não descrevem habilidade alguma.

In [2]:
habilidades = habilidades.loc[habilidades.name.notnull()]
print("Nº de registros restantes (com nome):", len(habilidades))

Nº de registros restantes (com nome): 2177


In [3]:
habilidades.head(10)

,endorsement_count,name,perfil_id
0,+ de 99,Empresas startups,0
3,+ de 99,Empreendedorismo,0
6,97,Start-ups,0
9,73,Entrepreneurship,0
10,75,New Business Development,0
11,67,Affiliate Marketing,0
12,67,Startup Development,0
13,64,Online Advertising,0
14,71,Business Strategy,0
15,68,Comércio eletrônico,0


## 04.2 - Contagem de Endossos e Habilidades

Afim de consolidar essa base em algum tipo de informação sintética por perfil, calcule a quantidade de habilidades (**qtd_habilidades**) e a quantidade total de endossos de cada perfil (**qtd_endossos**).

In [4]:
#Forma a resposta
#Cria as colunas: qtd_habilidades e qtd_endossos.
habilidades['qtd_habilidades']=1
habilidades['qtd_endossos']=habilidades.endorsement_count
habilidades.qtd_endossos.fillna('0', inplace=True)
habilidades.loc[habilidades.qtd_endossos.str.contains('de',case=False),'qtd_endossos']='100'
habilidades['qtd_endossos']=habilidades['qtd_endossos'].astype(int)
resposta04=habilidades.groupby('perfil_id').sum()

#Mostra o resultado
resposta04.head(10)

,qtd_habilidades,qtd_endossos
perfil_id,,
0,33,1600
1,28,314
2,47,117
3,9,14
4,11,103
5,29,127
6,23,126
7,5,0
8,50,1171


## 04.3 - Agrupando habilidades

Afim de entender melhor o perfil do funcionários no que tange diversidade, agrupe as habilidades em categorias. Por exemplo, c++, python em liguagens de programação, ferramentas do office informática, análise SWOT, PDCA, SCRUM gestão de projetos. Essa informação pode ser utilizada para averiguar a proximidades entre os perfis contratados em cada setor da empresa ou tipo cargo.

In [5]:
#Agrupando habilidades
habilidades['valor']=np.arange(len(habilidades.name))

#Vetores com as plavras-chaves para a função contains
v_empeinov=['startups','Start-ups', 'Startups','Empreendedorismo','Entrepreneurship',\
            'Innovation','Entrepreneur','Inovação','DAS','Start-up']
v_admeges=['Planejamento','Gestão','empresarial','projetos','Planning',\
           'Management','Leadership','Scrum','Estratégia','Resolução',\
           'Soluções','Melhoria','Seis','Lean','lean','Brand','Strategy','Gerenciamento',\
           'PDCA', 'SAP','Product','Teamwork','CEP','Metrologia','equipes','Organização',\
           'Project','Methodologies','PMI','Gerenciamento','Improvement','empresariais',\
           'Strategic','Análises','Consultoria','Contabilidade','Auditoria','Problem',\
           'Assessment','Psychometrics', 'Recruitments', 'Human Resources','CRM','Strategies',\
           'Palestras','Supply','5S','ERP','serviços','humanos','Treinamento','marca','Branding',\
           'Organizações','Metodologias','Recruiting','Consulting','Strategy','Coaching',\
           'Administração','Integração','PMO','Integration']
v_nemar=['Business','Comércio','Advertising','Marketing','Growth','Affiliate',\
         'Market','Finance','négocios','Publicidade','marketing','Pesquisa',\
         'Research','Cliente','Midia','financeira','bancários','Mercado',\
         'Negociação','Afiliadas','Customer','E-commerce','Financial','Commerce',\
         'Fusões','M&A','Stock','Capital','Mergers','Mídias','Afiliação','Investing',\
         'Investimentos','cliente','riscos','Seguros','Relações','Affiliation',\
         'Sales','Negotiation','Cartões','cards','mercado','Competitive','Finanças',\
         'Media','SEM','Advertising','PPC','Persona','Personas','negócios','Comunicação',\
         'Channels','Telecommunications','Análise da competitividade','Vendas','Marketing no Facebook',\
         'capitais','Mercados','Economia','Comunicação empresarial','Communicação estratégica',\
         'B2B','mercado','Salesforce.com','Precificação','Valuation','vendas','sales','atendimento',\
         'Call','Vendor','Insurance','Selling','venda','customer','Mídia']
v_desart=['Design','Interior','vocal','Edição','Sketching','Ilustração','Desenho','Music',\
          'Layout','Illustrator','Art','Video','Fotografia','Multimída','Creative',\
          'Film','Films','Compositing']
v_inf=['Google AdWords','Google','Adobe','Angular','Node','Git','AWS','AngularJS','CSS','Bootstrap',\
       'Google Analytics','WordPress','MySQL','MongoDB','Linux','Framework','Karma','Subversion','AMP',\
       'RequireJS','Gulp.js','GruntJS','Apache','PhoneGap','Mustache','Handlebars',\
       'Microsoft','AutoCad','PowerPoint','Scilab','WebTrends','Unix','SolidWorks',\
       'Windows','Android','Qt','Webkit','Mozilla','Maemo','GTK','Docker','JQuery',\
       'WebRTC','Electron','Photoshop','WebSockets', 'NoSQL', 'PostgreSQL', 'Web Services',\
       'Facebook','Scripting','Delphi','DOS','Mobile','Heroku','Bash','Amazon',\
       'Chrome','Eclipse','Compilers','CVS','Flask', 'Unix Shell Scripting', 'REST',\
       'Shell script','Sistemas distribuídos','Nginx', 'Django','Laravel', 'TDD',\
       'SOAP', 'Jenkins','Excel','Compatilidade','Ceph','Redis','Lua','Varnish','OpenSocial',\
       'Axure','mobile','CSS3','Wireframes','Sitemap','Copywriting','Tableau',\
       'Indesign','InDesign','E-learning', 'Moodle','Sabre','Amadeus','Oracle','Navitaire','Kanban','iOS',\
       'Swift','Selenium','JMeter','Fitnesse','eFitnesse','Cucumber','J2ME', 'Coded UI', 'Robotium',\
       'Monkeyrunner', 'Sikuli','Frameworks','Tétriz','Visual','TV','phpMyAdmin','EJB', 'JSF', 'JDBC',\
       'JPA','Ubuntu','J2ME','LaTeX','Jade','SML','FreeBSD','Debian','VMware','Pentaho','MS',\
       'Zabbix', 'Nagios','Hyperic','Runrun.it','Rancher','Silex','PHPUnit','Jasmine','Doctrine',\
       'Solve360','jQuery','Kubernetes','Multimídia','Foundation','JMUnit']
v_ti=['SEO','Online Lead','Sistemas','Systems','System','Web','User','Databases','Testing',\
      'Software','Promoter','Computer','Scalability','Information','dados','Data',\
      'Implementação','CLP','Computing','Network','Development','COBIT','Information',\
      'Front','Prototyping','Multithreading','computação','testes','sistemas','requisitos',\
      'informações','DevOps','IT','Hardware','Usuário','usuário','informação','Usability',\
      'Usabilidade','Accessibility','Responsiveness','Conversion','Page','Desenvolvimento',\
      'sistemas','TI','ITIL','development','Heuristic','Context','ETL','técnico',\
      'Tecnologias','front','Automação','Automotion','Bug', 'TestLink',\
      'bugs','BDD','Machine','DHTML','ITS','Database','BMC','Barman','Outsourcing',\
      'Infomation Retrieval','de rede','software','Dashboard','VBA','Performance','Teste',
      'Prototype','Test']
v_oraescr=['Speaking','Oratória','texto','Escrita','Redação']
v_lp=['JavaScript','TypeScript','HTML5','React','Redux','SASS','JQuery','HTML',\
      'PHP','OOP','UML','SQL','Socket.io','Algorithms','MapReduce','C','Java',\
      'Python','R','Ruby','.NET','XML','AJAX','Open','JSON','Programming',\
      'Perl','Escalabilidade', 'Algoritmo','Aberto','Programação','Go','ActionScript',\
      'LESS','Prolog','JavaSE','SQLite','UML','Pascal','Dojo']
v_eng=['Engenharia','Engineering','Engineering','aéreas','Metallurgy']
v_idi=['English','Inglês','Portuguese','Spanish','Abroad','Portugês','Portuguesa','Português'] 
v_dir=['Laws']
v_proa=['Fast','Proactivity','interpessoal','Capacidade','proativo','bom','Thinking','thinking','Visão',\
        'Proatividade', 'Persistência', 'Sociabilidade', 'Sensibilidade','Governance','Skills','Liderança',\
        'Bom']
v_edu=['Educacional','educacionais','Sports']
v_his=['History','Turism','Ethnography']
v_b=['Business Innovation']

def troca_string_pt1(vetor_string, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    r_vetor= [np.nan for x in range(len(df_controle))]
    for_vetor=df_controle.index
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                z=habilidades.loc[y,'valor']
                r_vetor[z]=valor
    return r_vetor

def troca_string_pt2(vetor_string, r_vetor, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    for_vetor=df_controle.index
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                z=habilidades.loc[y,'valor']
                r_vetor[z]=valor
    return r_vetor

#Cria a coluna: agrupamento
p1=troca_string_pt1(v_desart,habilidades.name,'Design / Artes')
p2=troca_string_pt2(v_lp,p1,habilidades.name,'Linguagem de Programação')
p3=troca_string_pt2(v_ti,p2,habilidades.name,'TI / Desenvolvedor')
p4=troca_string_pt2(v_inf,p3,habilidades.name,'Informática / Softwares')
p5=troca_string_pt2(v_admeges,p4,habilidades.name,'Gestão / Administração')
p6=troca_string_pt2(v_nemar,p5,habilidades.name,'Negócios / Marketing')
p7=troca_string_pt2(v_empeinov,p6,habilidades.name,'Empreendedorismo')
p6_1=troca_string_pt2(v_b,p7,habilidades.name,'Negócios / Marketing')
p8=troca_string_pt2(v_oraescr,p6_1,habilidades.name,'Oratória / Escrita')
p9=troca_string_pt2(v_eng,p8,habilidades.name,'Engenharia')
p10=troca_string_pt2(v_idi,p9,habilidades.name,'Idiomas')
p11=troca_string_pt2(v_proa,p10,habilidades.name,'Comportamento / Social')
p12=troca_string_pt2(v_edu,p11,habilidades.name,'Educação')
p13=troca_string_pt2(v_his,p12,habilidades.name,'História / Turismo')
habilidades['agrupamento']=troca_string_pt2(v_dir,p13,habilidades.name,'Direito')
habilidades.set_index('perfil_id')
habilidades.endorsement_count.fillna('Não Informado', inplace=True)

#Mostra o resultado
habilidades.head(10)

,endorsement_count,name,perfil_id,qtd_habilidades,qtd_endossos,valor,agrupamento
0,+ de 99,Empresas startups,0,1,100,0,Empreendedorismo
3,+ de 99,Empreendedorismo,0,1,100,1,Empreendedorismo
6,97,Start-ups,0,1,97,2,Empreendedorismo
9,73,Entrepreneurship,0,1,73,3,Empreendedorismo
10,75,New Business Development,0,1,75,4,Negócios / Marketing
11,67,Affiliate Marketing,0,1,67,5,Negócios / Marketing
12,67,Startup Development,0,1,67,6,TI / Desenvolvedor
13,64,Online Advertising,0,1,64,7,Negócios / Marketing
14,71,Business Strategy,0,1,71,8,Negócios / Marketing
15,68,Comércio eletrônico,0,1,68,9,Negócios / Marketing


## 04.4 - Salvando bases

A base tratada será gravada no arquivo **habilidades-tratado.csv**.

Uma nova base consolidada com um registro por perfil será gravada no arquivo **perfil-habilidades.csv**, contendo as colunas: **perfil_id**, **qtd_habilidades** e **qtd_endossos**.

In [6]:
# Grava a formação tratada em arquivo
habilidades_tratado=habilidades[['perfil_id','endorsement_count','name','agrupamento']].copy().set_index('perfil_id')
habilidades_tratado.to_csv('bases/habilidades-tratado.csv')

# Grava o consolidade da formação por perfil em arquivo
habilidades_perfil=resposta04.copy()
habilidades_perfil.to_csv('bases/perfil-habilidades.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)
* [Continuar para o próximo documento](05-tratamento-base-experiencia.ipynb)